<a href="https://colab.research.google.com/github/mostly-sunny/digital-health-hackathon/blob/main/temporary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [249]:
pip install pycox

In [250]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn_pandas import DataFrameMapper
import pandas as pd

import torch
import torchvision
import torchtuples as tt

from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

In [251]:
torch.cuda.is_available()

False

In [252]:
np.random.seed(123456)
torch.manual_seed(123456)
torch.cuda.manual_seed(123456)

In [253]:
dataset = pd.read_csv('/content/all-in-one-modified.csv')
dataset_for_hr = pd.read_csv('/content/test-data.csv')

In [254]:
dataset.head()

,Unnamed: 0,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38,G39,...,G274,G275,G276,G277,G278,G279,G280,G281,G282,G283,G284,G285,G286,G287,G288,G289,G290,G291,G292,G293,G294,G295,G296,G297,G298,G299,G300,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Treatment,time,event
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,1,4,6,5,2,1,0,1,0,57.448331,1
1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,3,9,1,1,2,4,0,1,0,27.004439,1
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,5,3,4,3,3,3,2,2,3,1,43.770511,1
3,3,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,2,7,2,3,5,0,1,4,5,3,1,32.281018,1
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,3,0,0,2,2,6,3,3,1,0,44.559284,0


In [255]:
dataset_test = dataset.sample(frac=0.3, random_state = np.random.seed(123456))
dataset_train = dataset.drop(dataset_test.index)
dataset_val = dataset_train.sample(frac=0.2, random_state = np.random.seed(123456))
dataset_train = dataset_train.drop(dataset_val.index)

In [256]:
columns_standardize = ['Var' + str(i) for i in range(1,11)]
columns_leave = ['G' + str(i) for i in range(1,301)]
columns_leave += ['Treatment']

standardize = [([col], MinMaxScaler()) for col in columns_standardize]

leave = [(col, None) for col in columns_leave]

x_mapper = DataFrameMapper(leave + standardize)

In [257]:
x_train = x_mapper.fit_transform(dataset_train).astype('float32')
x_val = x_mapper.transform(dataset_val).astype('float32')
x_test = x_mapper.transform(dataset_test).astype('float32')
x_for_hr = x_mapper.transform(dataset_for_hr).astype('float32')

In [258]:
get_target = lambda df: (df['time'].values, df['event'].values)
y_train = get_target(dataset_train)
y_val = get_target(dataset_val)

durations_test, events_test = get_target(dataset_test)
val = x_val, y_val

In [259]:
# def init_weights(m):
#     if type(m) == torch.nn.Linear:
#         torch.nn.init.xavier_uniform(m.weight)
#         m.bias.data.fill_(0.01)
#     # elif type(m) == torch.nn.ReLU:
#     #     torch.nn.init.xavier_uniform(m.weight)
#     #     m.bias.data.fill_(0.01)
#     elif type(m) == torch.nn.BatchNorm1d:
#         torch.nn.init.xavier_uniform(m.weight)
#         m.bias.data.fill_(0.01)

In [260]:
def make_net(in_features, out_features, hidden, nodes, rate):
  if hidden == 1:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(nodes, out_features)
    )
    # network.apply(init_weights)
  elif hidden == 2:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(nodes, int(nodes * rate)),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * rate)),
      torch.nn.Dropout(0.2),
            
      torch.nn.Linear(int(nodes * rate), out_features)
    )
  elif hidden == 3:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(nodes, int(nodes * rate)),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * rate)),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(int(nodes * rate), int(nodes * (rate ** 2))),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * (rate ** 2))),
      torch.nn.Dropout(0.2),
            
      torch.nn.Linear(int(nodes * (rate ** 2)), out_features)
    )
  elif hidden == 4:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(nodes, int(nodes * rate)),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * rate)),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(int(nodes * rate), int(nodes * (rate ** 2))),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * (rate ** 2))),
      torch.nn.Dropout(0.2),
      
      torch.nn.Linear(int(nodes * (rate ** 2)), int(nodes * (rate ** 3))),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * (rate ** 3))),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(int(nodes * (rate ** 3)), out_features)
    )
  return network

In [261]:
columns_list = ['brier', 'layers', 'nodes', 'rate', 'lr']
for i in range(1, 301):
  columns_list.append('G' + str(i))
result_rank = pd.DataFrame(columns = columns_list)
result_treat_ratio = pd.DataFrame(columns = columns_list)

In [262]:
result_treat_ratio

,brier,layers,nodes,rate,lr,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,...,G261,G262,G263,G264,G265,G266,G267,G268,G269,G270,G271,G272,G273,G274,G275,G276,G277,G278,G279,G280,G281,G282,G283,G284,G285,G286,G287,G288,G289,G290,G291,G292,G293,G294,G295,G296,G297,G298,G299,G300


In [263]:
result_rank

,brier,layers,nodes,rate,lr,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,...,G261,G262,G263,G264,G265,G266,G267,G268,G269,G270,G271,G272,G273,G274,G275,G276,G277,G278,G279,G280,G281,G282,G283,G284,G285,G286,G287,G288,G289,G290,G291,G292,G293,G294,G295,G296,G297,G298,G299,G300


In [264]:
in_features = x_train.shape[1]
out_features = 1

layers_list = [1]
nodes_list = [1024]
rate_list = [(1/2), (1/3)]
lr_list = [0.001]

total_num = len(layers_list) * len(nodes_list) * len(rate_list) * len(lr_list)
count = 1


for rate in rate_list:  
  for nodes in nodes_list:
    for layers in layers_list:
      for lr in lr_list:
        print(count, '/', total_num)
        print('layers:', layers, ', nodes:', nodes, ',rate:', rate, ',lr:', lr)
        count += 1

        # np.random.seed(123456)
        torch.manual_seed(123456)
        torch.cuda.manual_seed(123456)

        net = make_net(in_features, out_features, layers, nodes, rate)
        model = CoxPH(net, tt.optim.Adam)
        batch_size = len(dataset_train)

        if lr == 0:
          lrfinder = model.lr_finder(x_train, y_train, batch_size, tolerance = 10)
          model.optimizer.set_lr(lrfinder.get_best_lr())
        else:
          model.optimizer.set_lr(lr)
        
        epochs = 512
        callbacks = [tt.callbacks.EarlyStopping()]
        verbose = True

        %%time
        model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose, val_data=val, val_batch_size=batch_size)
        
        _ = model.compute_baseline_hazards()
        surv = model.predict_surv_df(x_test)

        # evaluation
        ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
        time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
        brier_score = ev.integrated_brier_score(time_grid)

        # calculate hazard ratio
        log_partial_hazard = model.predict(x_for_hr)
        partial_hazard = [np.exp(lph) for lph in log_partial_hazard]

        # ratio with treated and untreated
        treat_ratio = []
        for gene in range(300):
          treat_ratio.append([partial_hazard[gene+302]/partial_hazard[gene+2],'G' + str(gene+1)])
        
        treat_ratio_info = [float(i[0]) for i in treat_ratio]
        
        treat_ratio.sort()
        for i in range(300):
          treat_ratio[i][0] = i + 1
        treat_ratio.sort(key=lambda x:int(x[1][1:]))

        rank_info = [i[0] for i in treat_ratio]

        if lr == 0:
          info = [brier_score, layers, nodes, rate, lrfinder.get_best_lr()]
        else:
          info = [brier_score, layers, nodes, rate, lr]
        
        result_treat_ratio.loc[len(result_treat_ratio)] =  info + treat_ratio_info
        result_rank.loc[len(result_rank)] = info + rank_info

1 / 2
layers: 1 , nodes: 1024 ,rate: 0.5 ,lr: 0.001
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs
0:	[0s / 0s],		train_loss: 5.4958,	val_loss: 3.9256
1:	[0s / 0s],		train_loss: 5.1196,	val_loss: 3.9188
2:	[0s / 0s],		train_loss: 4.9200,	val_loss: 3.9117
3:	[0s / 0s],		train_loss: 4.7338,	val_loss: 3.9038
4:	[0s / 0s],		train_loss: 4.6017,	val_loss: 3.8954
5:	[0s / 1s],		train_loss: 4.4667,	val_loss: 3.8872
6:	[0s / 1s],		train_loss: 4.3412,	val_loss: 3.8790
7:	[0s / 1s],		train_loss: 4.2068,	val_loss: 3.8714
8:	[0s / 1s],		train_loss: 4.1298,	val_loss: 3.8646
9:	[0s / 1s],		train_loss: 4.0377,	val_loss: 3.8585
10:	[0s / 1s],		train_loss: 3.9434,	val_loss: 3.8526
11:	[0s / 1s],		train_loss: 3.8700,	val_loss: 3.8468
12:	[0s / 1s],		train_loss: 3.7924,	val_loss: 3.8418
13:	[0s / 2s],		train_loss: 3.7284,	val_loss: 3.8363
14:	[0s / 2s],		train_loss: 3.6584,	val_loss: 3.8309
15:	[0s / 2s],		train_loss: 3.6251,	val_loss: 3.8266
16:	[0s / 2s],		train_loss: 3.5661,	val_loss: 

In [265]:
result_treat_ratio

,brier,layers,nodes,rate,lr,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,...,G261,G262,G263,G264,G265,G266,G267,G268,G269,G270,G271,G272,G273,G274,G275,G276,G277,G278,G279,G280,G281,G282,G283,G284,G285,G286,G287,G288,G289,G290,G291,G292,G293,G294,G295,G296,G297,G298,G299,G300
0,0.056082,1.0,1024.0,0.500000,0.001,0.736768,0.769267,0.761322,0.758448,0.746528,0.719237,0.781001,0.762144,0.744,0.743725,0.709154,0.739925,0.739428,0.73005,0.742602,0.748932,0.706708,0.733467,0.724367,0.752104,0.725208,0.744706,0.756921,0.739759,0.741146,0.744281,0.753945,0.747261,0.728698,0.74906,0.7321,0.745195,0.719673,0.726619,0.709577,...,0.731864,0.760957,0.733371,0.7332,0.76115,0.733097,0.75913,0.743232,0.747959,0.714692,0.73123,0.747952,0.72961,0.729529,0.70595,0.722202,0.760191,0.710932,0.744185,0.725217,0.729821,0.735692,0.755255,0.708322,0.7419,0.716075,0.747662,0.761874,0.740605,0.737422,0.723785,0.728552,0.737862,0.741026,0.732405,0.725209,0.736437,0.744078,0.72929,0.765038
1,0.056082,1.0,1024.0,0.333333,0.001,0.736768,0.769267,0.761322,0.758448,0.746528,0.719237,0.781001,0.762144,0.744,0.743725,0.709154,0.739925,0.739428,0.73005,0.742602,0.748932,0.706708,0.733467,0.724367,0.752104,0.725208,0.744706,0.756921,0.739759,0.741146,0.744281,0.753945,0.747261,0.728698,0.74906,0.7321,0.745195,0.719673,0.726619,0.709577,...,0.731864,0.760957,0.733371,0.7332,0.76115,0.733097,0.75913,0.743232,0.747959,0.714692,0.73123,0.747952,0.72961,0.729529,0.70595,0.722202,0.760191,0.710932,0.744185,0.725217,0.729821,0.735692,0.755255,0.708322,0.7419,0.716075,0.747662,0.761874,0.740605,0.737422,0.723785,0.728552,0.737862,0.741026,0.732405,0.725209,0.736437,0.744078,0.72929,0.765038


In [266]:
result_rank

,brier,layers,nodes,rate,lr,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,...,G261,G262,G263,G264,G265,G266,G267,G268,G269,G270,G271,G272,G273,G274,G275,G276,G277,G278,G279,G280,G281,G282,G283,G284,G285,G286,G287,G288,G289,G290,G291,G292,G293,G294,G295,G296,G297,G298,G299,G300
0,0.056082,1.0,1024.0,0.500000,0.001,145.0,291.0,280.0,267.0,215.0,39.0,298.0,282.0,192.0,190.0,11.0,163.0,156.0,94.0,184.0,229.0,6.0,120.0,53.0,246.0,60.0,202.0,259.0,159.0,178.0,199.0,251.0,219.0,83.0,230.0,108.0,205.0,41.0,68.0,12.0,...,106.0,278.0,118.0,116.0,279.0,114.0,270.0,188.0,224.0,28.0,103.0,223.0,91.0,90.0,5.0,45.0,277.0,16.0,196.0,62.0,92.0,134.0,253.0,10.0,180.0,31.0,222.0,281.0,167.0,147.0,50.0,81.0,151.0,174.0,109.0,61.0,139.0,195.0,85.0,288.0
1,0.056082,1.0,1024.0,0.333333,0.001,145.0,291.0,280.0,267.0,215.0,39.0,298.0,282.0,192.0,190.0,11.0,163.0,156.0,94.0,184.0,229.0,6.0,120.0,53.0,246.0,60.0,202.0,259.0,159.0,178.0,199.0,251.0,219.0,83.0,230.0,108.0,205.0,41.0,68.0,12.0,...,106.0,278.0,118.0,116.0,279.0,114.0,270.0,188.0,224.0,28.0,103.0,223.0,91.0,90.0,5.0,45.0,277.0,16.0,196.0,62.0,92.0,134.0,253.0,10.0,180.0,31.0,222.0,281.0,167.0,147.0,50.0,81.0,151.0,174.0,109.0,61.0,139.0,195.0,85.0,288.0
